In [ ]:
#creating spark instance
from pyspark.sql import SparkSession
spark = SparkSession .builder.appName('removeduplicatescart').getOrCreate()

In [ ]:
#import libraries
import pandas as pd
import numpy as np
from pyspark.sql import functions as sf

In [ ]:
#Read column names
headers1 = spark.read.csv("sample1.csv",inferSchema =True, header=True)
#headers1.printSchema()

In [ ]:
#Read data from tsv file
rows = spark.read.option("sep", "\t").csv("..\hit_data.tsv",inferSchema =True, header=False)
#rows.printSchema()

In [ ]:
#adding header to data
rowsWithHeader1 = headers1.union(rows)
#rowsWithHeader1.printSchema()
rowsWithHeader1.count()

In [ ]:
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.exclude_hit<=0)
rowsCount = rowsWithHeader1.count()
rowsCount

In [ ]:
#filtering rows based on hit_source column value. Exclude all rows with hit_source = 5,7,8,9.
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=5)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=7)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=8)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=9)

In [ ]:
#Selecting only post columns
postData = rowsWithHeader1.select(*filter(lambda col: 'post_' in col,rowsWithHeader1.columns))
#Printing no. of columns
len(postData.columns)

In [ ]:
#Add a new column which conains values to idenify unique visitors
postDataWithUniqueId = postData.withColumn('joined_column', sf.concat(sf.col('post_visid_high'),sf.lit(''), sf.col('post_visid_low')))

In [ ]:
#apart from post_ columns considering column obtained after joining the two post_ columns as mentioned above and mcvisid
postDataWithUniqueId = postDataWithUniqueId.withColumnRenamed('joined_column', 'post_uniqueId')

In [ ]:
#Select the column post_uniqueID
UniqueId=postDataWithUniqueId.select('post_uniqueId')
#Print count
UniqueId.count()

In [ ]:
postDataWithUniqueId.schema

In [ ]:
#Removing post_mobile columns from dataframe
#postDataWithoutMobile = postDataWithUniqueId.drop(*filter(lambda col: 'mobile' in col,postData.columns))
#Printing no. of columns
#len(postDataWithoutMobile.columns)

In [ ]:
#select users with cart,ship,chkout4_confirm in referrer
#cartShipConfirmRows = postDataWithoutMobile.filter(postDataWithoutMobile.post_referrer.rlike("^(.)*(cart|CART|Cart|shipping|Ship|ship|chkout4_confirm)(.)*$"))
#cartShipConfirmRows.count()

In [ ]:
#Edit

In [ ]:
#Edit
#select users with cart,ship,chkout4_confirm in referrer
cartDellRows = postDataWithUniqueId.filter(postDataWithUniqueId.post_prop14.rlike("^(.)*(dell.com)(.)*(cart)(.)*$"))
cartDellRows.count()

In [ ]:
#storing all existing distinct users
uniqueIdsWithCartDell = [list(x.asDict().values())[0] for x in cartDellRows.select("post_uniqueid").distinct().collect()]
print("{}".format(len(uniqueIdsWithCartDell)))

In [ ]:
#Get all rows of users having cart, ship, chkout4_confirm in post_referrer
allRowsOfCartDellUsers = postDataWithUniqueId.where(postDataWithUniqueId.post_uniqueId.isin(uniqueIdsWithCartDell))
#Print count
allRowsOfCartDellUsersCount = allRowsOfCartDellUsers.count()
allRowsOfCartDellUsersCount

In [ ]:
#Regex to find event list having 1 at beginning, in middle and at end.
event1CartDellRows = allRowsOfCartDellUsers.filter(allRowsOfCartDellUsers.post_event_list.rlike("^(1,[0-9 , . =]*|[0-9 , . =]*,1,[0-9 , . =]*|[0-9 , . =]*,1)$"))
#print count
event1CartDellRows.count()

In [ ]:
#Get unique Ids
event1CartDellUniqueIds = [list(x.asDict().values())[0] for x in event1CartDellRows.select("post_uniqueid").distinct().collect()]
#print count
print("{}".format(len(event1CartDellUniqueIds)))

In [ ]:
#Get abandoned userIds
abandonedCartDellUsersIds = list(set(uniqueIdsWithCartDell) - set(event1CartDellUniqueIds))
#print count
print("{}".format(len(abandonedCartDellUsersIds)))

In [ ]:
#Reducing data by removing null columns
allRowsOfCartDellUsersPDF = allRowsOfCartDellUsers.toPandas()
allRowsOfCartDellUsersPDF.shape

In [ ]:
#Edit

In [ ]:
#Removes any columns with null values
rowsWithoutNullColumns = allRowsOfCartDellUsersPDF.dropna(axis=1,how='all')
rowsWithoutNullColumns.shape

In [ ]:
#Removing columns with 50% nulls 
rowsWithoutNullColumnsThresh50 = rowsWithoutNullColumns.dropna(axis=1,thresh=allRowsOfCartDellUsersCount/2)
rowsWithoutNullColumnsThresh50.shape

In [ ]:
#test50 = rowsWithoutNullColumnsThresh50.copy()
#test50.loc[test50['post_uniqueId'].str.isdigit(), 'post_uniqueId'] = pd.to_numeric(test50['post_uniqueId'], errors='coerce')
#test50['post_uniqueId'] = test50['post_uniqueId'].astype(np.float64)
#test50['post_uniqueId'] = test50['post_uniqueId'].astype(str)
#test50.dtypes
#test50.to_csv("test50.csv", index=False)

In [ ]:
#Save dataframe to a csv file
#rowsWithoutNullColumnsThresh50.to_csv("reducedusersrows.csv", index=False)
#Save dataframe to a tsv file
rowsWithoutNullColumnsThresh50.to_csv("version1/reducedusersrows.tsv", sep="\t", index=False)

In [ ]:
#Read csv file
#allUsers = spark.read.csv("reducedusersrows.csv",inferSchema =True, header=True)
#Read tsv file
allUsers = spark.read.option("sep", "\t").csv("version1/reducedusersrows.tsv",inferSchema =True, header=True)
#allUsers.printSchema()

In [ ]:
allUsers.schema.names

In [ ]:
#get all rows of abandoned users
allRowsOfAbandonedUsers = allUsers.where(allUsers.post_uniqueId.isin(abandonedCartDellUsersIds))
#print count
allRowsOfAbandonedUsers.count()

In [ ]:
#Groupby abandoned users using post_uniqueId
abandonedUsersGroupBy = allRowsOfAbandonedUsers.groupBy("post_uniqueId").count()

In [ ]:
abandonedUsersGroupBy.show()

In [ ]:
#get all rows of purchased users
allRowsOfPurchasedUsers = allUsers.where(allUsers.post_uniqueId.isin(event1CartDellUniqueIds))
#print count
allRowsOfPurchasedUsers.count()

In [ ]:
#Groupby purchased users using post_uniqueId
purchasedUsersGroupBy = allRowsOfPurchasedUsers.groupBy("post_uniqueId").count()

In [ ]:
purchasedUsersGroupBy.show()

In [ ]:
#Displaying column names
#allRowsOfPurchasedUsers.schema.names

In [ ]:
#Selecting post_prop14 and post_referrer columns
allRowsOfPurchasedUsers.select("post_prop14","post_referrer").show()

In [ ]:
#Selecting evar34 and evar35 columns
allRowsOfPurchasedUsers.select("post_evar34","post_evar35").show()

In [ ]:
#Displaying schema
#allRowsOfPurchasedUsers.schema

In [ ]:
#Convert to pandas Dataframe
purchasedPDF = allRowsOfPurchasedUsers.toPandas()

In [ ]:
#print
purchasedPDF

In [ ]:
#purchasedPDF1 = pd.concat([purchasedPDF[col].dropna().reset_index(drop=True) for col in purchasedPDF], axis=1)

In [ ]:
#print column datatypes
purchasedPDF.dtypes

In [ ]:
#converting columns to string type Not Working
#purchasedPDF.astype(str)
#purchasedPDF.dtypes

In [ ]:
#fill na values with none
purchasedPDFFill = purchasedPDF.fillna('none')

In [ ]:
#Convert all columns to string type
purchasedPDFFillString = purchasedPDFFill.applymap(str)
purchasedPDFFillString.dtypes

In [ ]:
purchasedSingleRow = pd.DataFrame()

In [ ]:
#Storing uniqueid
purchasedSingleRow['post_uniqueId'] = purchasedPDFFill.groupby(['post_uniqueId'])['post_uniqueId'].count().index.get_level_values('post_uniqueId')
#purchasedSingleRow is a dataframe containing post_uniqueId column

In [ ]:
type(purchasedSingleRow)

In [ ]:
purchasedSingleRow

In [ ]:
#purchasedSingleRow.drop('index', axis = 1, inplace=True)

In [ ]:
for cols in purchasedPDFFillString.columns:
    if (cols != 'post_uniqueId'):
        temp = purchasedPDFFillString.groupby(['post_uniqueId'])[cols].apply(lambda x: '~'.join(x)).reset_index()
        purchasedSingleRow[cols] = temp[cols]

In [ ]:
purchasedSingleRow

In [ ]:
type(purchasedSingleRow)

In [ ]:
#Convert to pandas Dataframe
abandonedPDF = allRowsOfAbandonedUsers.toPandas()

In [ ]:
#print
abandonedPDF

In [ ]:
#purchasedPDF1 = pd.concat([purchasedPDF[col].dropna().reset_index(drop=True) for col in purchasedPDF], axis=1)

In [ ]:
#print column datatypes
abandonedPDF.dtypes

In [ ]:
#fill na values with none
abandonedPDFFill = abandonedPDF.fillna('none')

In [ ]:
#Convert all columns to string type
abandonedPDFFillString = abandonedPDFFill.applymap(str)
abandonedPDFFillString.dtypes

In [ ]:
abandonedSingleRow = pd.DataFrame()

In [ ]:
#Storing uniqueid
abandonedSingleRow['post_uniqueId'] = abandonedPDFFill.groupby(['post_uniqueId'])['post_uniqueId'].count().index.get_level_values('post_uniqueId')
#purchasedSingleRow is a dataframe containing post_uniqueId column

In [ ]:
type(abandonedSingleRow)

In [ ]:
abandonedSingleRow

In [ ]:
#purchasedSingleRow.drop('index', axis = 1, inplace=True)

In [ ]:
for cols in abandonedPDFFillString.columns:
    if (cols != 'post_uniqueId'):
        temp = abandonedPDFFillString.groupby(['post_uniqueId'])[cols].apply(lambda x: '~'.join(x)).reset_index()
        abandonedSingleRow[cols] = temp[cols]

In [ ]:
abandonedSingleRow